In [ ]:
import pandas as pd
from clingo import Control
import ast
from matching_utils import get_invalid_matches, check_mentor_capacity

students_df =pd.read_csv('../data/studenten.csv')
mentors_df =pd.read_csv('../data/mentoren.csv')

education_mapping = {
    'Associate': 1,
    'Bachelor': 2, 
    'Master': 3,
    'PhD': 4
}

def generate_asp_facts():
    facts = []
    
    for _, row in students_df.iterrows():
        student_id = f"s{row.name}"  
        facts.append(f"student({student_id}).")
        
        edu_level = education_mapping[row['Opleidingsniveau']]
        facts.append(f"education({student_id}, {edu_level}).")
        
        subject = row['Onderwerp'].lower().replace(' ', '_')
        facts.append(f"expertise({student_id}, {subject}).")

        availability = ast.literal_eval(row['Beschikbaarheid'])
        for day in availability:
            day_clean = day.lower()
            facts.append(f"availability({student_id}, {day_clean}).")
    
    for _, row in mentors_df.iterrows():
        mentor_id = f"m{row.name}"  
        facts.append(f"mentor({mentor_id}).")
        
        edu_level = education_mapping[row['Opleidingsniveau']]
        facts.append(f"education({mentor_id}, {edu_level}).")
        
        subjects = ast.literal_eval(row['Onderwerpen'])
        for subject in subjects:
            subject_clean = subject.lower().replace(' ', '_')
            facts.append(f"expertise({mentor_id}, {subject_clean}).")

        max_students = row['Max_Studenten']
        facts.append(f"max_students({mentor_id}, {max_students}).")

        availability = ast.literal_eval(row['Beschikbaarheid'])
        for day in availability:
            day_clean = day.lower()
            facts.append(f"availability({mentor_id}, {day_clean}).")
    
    return "\n".join(facts)

asp_facts = generate_asp_facts()

ctl = Control()
ctl.add("base", [], f"""
{asp_facts}

% Matching constraints 
% A mentor M is a valid mentor for a student S if:
%     S is a student,
%     M is a mentor,
%     The expertise of M is Subj,
%     The expertise of S is Subj, (the same)
%     The education of M is EM,
%     The education of S is ES,
%     EM > ES.
%     The availability of S is Day,
%     The availability of M is Day, (the same)

valid_mentor(S, M) :-
    student(S),
    mentor(M), 
    expertise(M, Subj), 
    expertise(S, Subj), 
    education(M, EM), 
    education(S, ES), 
    EM > ES,
    availability(S, Day),
    availability(M, Day).

% ONLY allow valid matches no invalid matches allowed (constraint)
:- match_mentor(S, M), not valid_mentor(S, M).

% Allow 0 or 1 mentor per student 
{{ match_mentor(S, M) : valid_mentor(S, M) }} <= 1 :- student(S).

% Allow max max_students per mentors
{{ match_mentor(S, M) : student(S) }} <= Max :- mentor(M), max_students(M, Max).

% Maximize the number of matched students 
#maximize {{ 1,S : match_mentor(S,M) }}.

% Output
#show match_mentor/2.
""")

ctl.ground([("base", [])])

def make_matches():
    matches = []
    
    def collect_matches(model):
        nonlocal matches
        matches = []
        
        for symbol in model.symbols(shown=True):
            if symbol.name == "match_mentor":
                student_id = symbol.arguments[0].name
                mentor_id = symbol.arguments[1].name
                
                student_idx = int(student_id[1:])  
                mentor_idx = int(mentor_id[1:])    
                
                student_data = {
                    'voornaam': students_df.iloc[student_idx]['Voornaam'],
                    'achternaam': students_df.iloc[student_idx]['Achternaam'],
                    'opleidingsniveau': students_df.iloc[student_idx]['Opleidingsniveau'],
                    'onderwerp': students_df.iloc[student_idx]['Onderwerp'],
                    'beschikbaarheid': ast.literal_eval(students_df.iloc[student_idx]['Beschikbaarheid'])
                }
                
                mentor_data = {
                    'voornaam': mentors_df.iloc[mentor_idx]['Voornaam'],
                    'achternaam': mentors_df.iloc[mentor_idx]['Achternaam'],
                    'opleidingsniveau': mentors_df.iloc[mentor_idx]['Opleidingsniveau'],
                    'onderwerpen': ast.literal_eval(mentors_df.iloc[mentor_idx]['Onderwerpen']),
                    'max_studenten': mentors_df.iloc[mentor_idx]['Max_Studenten'],
                    'beschikbaarheid': ast.literal_eval(mentors_df.iloc[mentor_idx]['Beschikbaarheid'])
                }
                
                matches.append((student_data, mentor_data))
    
    result = ctl.solve(on_model=collect_matches)
    
    if result.satisfiable:
        return matches
    else:
        return []

# Usage
matches = make_matches()
print(matches)
print(len(matches))
print(get_invalid_matches(matches))
print(check_mentor_capacity(matches))
# for student, mentor in matches:
#     print(f"{student['voornaam']} {student['achternaam']} ({student['opleidingsniveau']}, {student['onderwerp']}) → {mentor['voornaam']} {mentor['achternaam']} ({mentor['opleidingsniveau']}, {mentor['onderwerpen']})")

[({'voornaam': 'Leah', 'achternaam': "van 't Riet", 'opleidingsniveau': 'Bachelor', 'onderwerp': 'Ethical Hacking', 'beschikbaarheid': "['Maandag']"}, {'voornaam': 'Xavi', 'achternaam': 'van Wickerode', 'opleidingsniveau': 'Master', 'onderwerpen': ['Ethical Hacking', 'Data Visualisation', 'Software Architecture', 'Business Process Analytics'], 'max_studenten': np.int64(5), 'beschikbaarheid': "['Maandag', 'Woensdag', 'Donderdag', 'Zaterdag']"}), ({'voornaam': 'Damian', 'achternaam': 'Gnodde', 'opleidingsniveau': 'Bachelor', 'onderwerp': 'Ethical Hacking', 'beschikbaarheid': "['Maandag', 'Dinsdag', 'Woensdag', 'Vrijdag', 'Zondag']"}, {'voornaam': 'Xavi', 'achternaam': 'van Wickerode', 'opleidingsniveau': 'Master', 'onderwerpen': ['Ethical Hacking', 'Data Visualisation', 'Software Architecture', 'Business Process Analytics'], 'max_studenten': np.int64(5), 'beschikbaarheid': "['Maandag', 'Woensdag', 'Donderdag', 'Zaterdag']"}), ({'voornaam': 'Elizabeth', 'achternaam': 'van Kuijc', 'opleidi

In [ ]:
def debug_matching():
    
    matches = make_matches()
    print(f"Total matches found: {len(matches)}")
    
    invalid_matches = get_invalid_matches(matches)
    capacity_violations = check_mentor_capacity(matches)
    
    print(f"Invalid matches (education/expertise): {len(invalid_matches)}")
    print(f"Capacity violations: {len(capacity_violations)}")
    
    if invalid_matches:
        print("\nInvalid matches found:")
        for student, mentor in invalid_matches[:3]:
            print(f"  {student['voornaam']} {student['achternaam']} ({student['opleidingsniveau']}, {student['onderwerp']})")
            print(f"  : {mentor['voornaam']} {mentor['achternaam']} ({mentor['opleidingsniveau']}, {mentor['onderwerpen']})")
            print()
    
    if capacity_violations:
        print(f"Mentors over capacity: {capacity_violations}")

debug_matching()

Total matches found: 100
Invalid matches (education/expertise): 0
Capacity violations: 0
